In [78]:
import pandas as pd

In [79]:
with open('onto.tsv', 'r') as f:
    #onto = pd.read_csv(f, sep='\t', header=0)
    text = f.read()

# RE


In [80]:
import re
from re import match

### class_to_pred

In [81]:
s = "MemberOf(x,Plant)"
a = re.sub(r'(MemberOf)(\()(\w*)(,)(\w*)(\))', r'\5\2\3\6',s)
a

'Plant(x)'

In [82]:
def class_to_pred(s):
    s = re.sub(r'(MemberOf)(\()(\\|\s|&)*(\w*)(,)(\\|\s|&)*(\w*)(\\|\s|&)*(\))', r'\7\2\4\9',s)
    return s

In [83]:
class_to_pred("MemberOf( \\ &                wind, \\ &                InanimateCarrier \\ &                )")

'InanimateCarrier(wind)'

In [84]:
text = class_to_pred(text)

### Disjoint Union

In [85]:
def partition_to_disjunion(text):
    p = re.compile(r'Partition\((\\|\s|&)*\((\\|\s|&)*((\\*|\s*|&|\w+|,|\s*)*)\.*\)(\\|\s|&)*,(\\|\s|&)*(\w+)(\\|\s|&)*\)')
    regex = re.compile(p)
    for orig_partition in regex.finditer(text):
        orig_partition = orig_partition.group(0)
        print(orig_partition)
        partitions = p.match(orig_partition).group(3).replace(' ','').split(',')
        print(partitions)
        parent = p.match(orig_partition).group(7)
        print(parent)

        a_parent = str('{bold_begin}a:'+parent+'{bold_end}').format(bold_begin='\\code{', bold_end='}')
        a_partitions = str(['\\& '+x for x in str([str('{bold_begin}a:'+part+'{bold_end}').format(bold_begin='\\code{', bold_end='}') for part in partitions]).replace('[','').replace(']','').replace('\'','').replace('&','').replace('\\','').replace('a: ',' ').split(',')]).replace('\'','').replace('[','').replace(']','').replace(',','').replace(' bm','\\bm')#.replace('\\\\','\\')

        disj_union = '{bold_begin}DisjointUnion({bold_end}\\\\& {a_parent} {a_partitions}\\\\&{bold_begin}){bold_end}'.format(bold_begin='\\code{', bold_end='}', a_parent=a_parent, a_partitions=a_partitions)
        
        text = text.replace(orig_partition, disj_union)

    return text

In [86]:

s = "Partition(&(\\Flower, \\ Tree, ...),Plant)      \±jfid Partition((Sun, ...),Star)"
partition_to_disjunion(s)

Partition(&(\Flower, \ Tree, ...),Plant)
['Flower', '\\Tree', '']
Plant
Partition((Sun, ...),Star)
['Sun', '']
Star


'\\code{DisjointUnion(}\\\\& \\code{a:Plant} \\\\& code{a:Flower} \\\\&  code{a:Tree} \\\\&  code{a:}\\\\&\\code{)}      \\±jfid \\code{DisjointUnion(}\\\\& \\code{a:Star} \\\\& code{a:Sun} \\\\&  code{a:}\\\\&\\code{)}'

In [87]:
text = partition_to_disjunion(text)

Partition( \\ &                (Organism,\\ &InanimateEntity), \\ &                PathogenCarrier \\ &                )
['Organism', '\\\\&InanimateEntity']
PathogenCarrier
Partition( \\ &                (VectingCarrier,\\ &NonVectingCarrier), \\ &                PathogenCarrier \\ &                )
['VectingCarrier', '\\\\&NonVectingCarrier']
PathogenCarrier
Partition( \\ &                (Organism,\\ &InanimateEntity), \\ &                PathogenCarrier \\ &                )
['Organism', '\\\\&InanimateEntity']
PathogenCarrier
Partition( \\ &                (LivingOrganism,\\ &NonLivingOrganism), \\ &                Organism \\ &                )
['LivingOrganism', '\\\\&NonLivingOrganism']
Organism
Partition( \\ &                (Bacterium,\\ &Fungus,\\ &Virus), \\ &                Pathogen \\ &                )
['Bacterium', '\\\\&Fungus', '\\\\&Virus']
Pathogen
Partition( \\ &                (Trophic,\\ &NonTrophic), \\ &                BiologicalInteraction \\ &               

## Save File

In [88]:
with open('onto_out.tsv', 'w+') as f:
    #onto = pd.read_csv(f, sep='\t', header=0)
    f.write(text)